see paper: https://arxiv.org/pdf/1907.05576

# NAML: Neural News Recommendation with Attentive Multi-View Learning

In [1]:
from pathlib import Path
import tensorflow as tf
import polars as pl
import numpy as np
import os

from ebrec.utils._constants import *
from ebrec.utils._behaviors import create_binary_labels_column, sampling_strategy_wu2019, add_known_user_column, add_prediction_scores, truncate_history
from ebrec.utils._articles import convert_text2encoding_with_transformers, create_article_id_to_value_mapping
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._nlp import get_transformers_word_embeddings

from transformers import AutoTokenizer, AutoModel


"""
load data
"""

data_base = Path(os.getcwd()).parent / "data-merged" / "merged"
# train_val_base = data_base / "1-ebnerd_demo_(20MB)"
train_val_base = data_base / "2-ebnerd_small_(80MB)"
# train_val_base = data_base / "3-ebnerd_large_(3.0GB)"
test_base = data_base / "5-ebnerd_testset_(1.5GB)"
assert train_val_base.exists() and test_base.exists()

train_behaviors = pl.scan_parquet(train_val_base / "train" / "behaviors.parquet")
train_history = pl.scan_parquet(train_val_base / "train" / "history.parquet")

val_behavior = pl.scan_parquet(train_val_base / "validation" / "behaviors.parquet")
val_history = pl.scan_parquet(train_val_base / "validation" / "history.parquet")

test_behavior = pl.scan_parquet(test_base / "test" / "behaviors.parquet")
test_history = pl.scan_parquet(test_base / "test" / "history.parquet")

train_articles: pl.LazyFrame = pl.scan_parquet(train_val_base / "articles.parquet")
val_articles: pl.LazyFrame = train_articles
test_articles: pl.LazyFrame = pl.scan_parquet(test_base / "articles.parquet")

articles_word2vec: pl.LazyFrame = pl.scan_parquet(data_base / "7-Ekstra-Bladet-word2vec_(133MB)" / "document_vector.parquet")
articles_image_embeddings: pl.LazyFrame = pl.scan_parquet(data_base / "8-Ekstra_Bladet_image_embeddings_(372MB)" / "image_embeddings.parquet")
articles_contrastive_vector: pl.LazyFrame = pl.scan_parquet(data_base / "9-Ekstra-Bladet-contrastive_vector_(341MB)" / "contrastive_vector.parquet")
articles_bert_base_multilingual_cased: pl.LazyFrame = pl.scan_parquet(data_base / "10-google-bert-base-multilingual-cased_(344MB)" / "bert_base_multilingual_cased.parquet")
articles_xlm_roberta_base: pl.LazyFrame = pl.scan_parquet(data_base / "11-FacebookAI-xlm-roberta-base_(341MB)" / "xlm_roberta_base.parquet")


"""
preprocessing: truncate user history, select subset of columns, join behavior and history, sample based on Wu2019, add binary labels column
"""


def ebnerd_from_path(history: pl.LazyFrame, behaviors: pl.LazyFrame, history_size: int = 30) -> pl.DataFrame:
    df_history = history.select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL).pipe(truncate_history, column=DEFAULT_HISTORY_ARTICLE_ID_COL, history_size=history_size, padding_value=0)
    df_behaviors = behaviors.collect().pipe(slice_join_dataframes, df2=df_history.collect(), on=DEFAULT_USER_COL, how="left")
    return df_behaviors


COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
]
HISTORY_SIZE = 30
N_SAMPLES = 100
df_train = (
    ebnerd_from_path(history=train_history, behaviors=train_behaviors, history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(sampling_strategy_wu2019, npratio=4, shuffle=True, with_replacement=True, seed=123)
    .pipe(create_binary_labels_column)
    .sample(n=N_SAMPLES)
)
df_validation = ebnerd_from_path(history=val_history, behaviors=val_behavior, history_size=HISTORY_SIZE).select(COLUMNS).pipe(create_binary_labels_column).sample(n=N_SAMPLES)
df_test = (
    ebnerd_from_path(history=test_history, behaviors=val_behavior, history_size=HISTORY_SIZE)
    .with_columns(pl.Series(DEFAULT_CLICKED_ARTICLES_COL, [[]]))
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    .sample(n=N_SAMPLES)
)


"""
use huggingface transformers to convert article text to tokens, tokens to embeddings
"""

df_articles = train_articles.collect()
TRANSFORMER_MODEL_NAME = "bert-base-multilingual-cased"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

word2vec_embedding = get_transformers_word_embeddings(transformer_model)
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH)
article_mapping = create_article_id_to_value_mapping(df=df_articles, value_col=token_col_title)

/Users/sueszli/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/ebrec/utils/_behaviors.py:619: UserWarning: truncate_history: The history IDs expeced in ascending order
  warnings.warn(f"{function_name}: The history IDs expeced in ascending order")
/Users/sueszli/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/sueszli/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
"""
batch data
"""
from ebrec.models.newsrec.dataloader import NAMLDataLoader

DEFAULT_ARTICLE_MODIFIED_TIMESTAMP_COL = "last_modified_time"
DEFAULT_ARTICLE_PUBLISHED_TIMESTAMP_COL = "published_time"
DEFAULT_SENTIMENT_LABEL_COL = "sentiment_label"
DEFAULT_SENTIMENT_SCORE_COL = "sentiment_score"
DEFAULT_TOTAL_READ_TIME_COL = "total_read_time"
DEFAULT_TOTAL_PAGEVIEWS_COL = "total_pageviews"
DEFAULT_TOTAL_INVIEWS_COL = "total_inviews"
DEFAULT_ARTICLE_TYPE_COL = "article_type"
DEFAULT_CATEGORY_STR_COL = "category_str"
DEFAULT_SUBCATEGORY_COL = "subcategory"
DEFAULT_ENTITIES_COL = "entity_groups"
DEFAULT_IMAGE_IDS_COL = "image_ids"
DEFAULT_SUBTITLE_COL = "subtitle"
DEFAULT_CATEGORY_COL = "category"
DEFAULT_NER_COL = "ner_clusters"
DEFAULT_PREMIUM_COL = "premium"
DEFAULT_TOPICS_COL = "topics"
DEFAULT_TITLE_COL = "title"
DEFAULT_BODY_COL = "body"
DEFAULT_URL_COL = "url"


user_id_mapping: dict[int, int] = {user_id: i for i, user_id in enumerate(df_train[DEFAULT_USER_COL].unique())}
body_mapping: dict[int, list[int]] = {article_id: embeddings for article_id, embeddings in zip(df_articles[DEFAULT_ARTICLE_ID_COL], df_articles[token_col_title])}
category_mapping: dict[int, int] = {article_id: category for article_id, category in zip(df_articles[DEFAULT_ARTICLE_ID_COL], df_articles[DEFAULT_CATEGORY_COL])}
subcategory_mapping: dict[int, int] = {article_id: subcategory for article_id, subcategory in zip(df_articles[DEFAULT_ARTICLE_ID_COL], df_articles[DEFAULT_SUBCATEGORY_COL])}


train_dataloader = NAMLDataLoader(
    body_mapping=body_mapping,
    category_mapping=category_mapping,
    subcategory_mapping=subcategory_mapping,

    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    batch_size=64,
)
val_dataloader = NAMLDataLoader(
    body_mapping=body_mapping,
    category_mapping=category_mapping,
    subcategory_mapping=subcategory_mapping,

    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    batch_size=32,
)
test_dataloader = NAMLDataLoader(
    body_mapping=body_mapping,
    category_mapping=category_mapping,
    subcategory_mapping=subcategory_mapping,

    behaviors=df_test,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    batch_size=32,
)

In [5]:
"""
train model
"""
from ebrec.models.newsrec.model_config import hparams_npa
from ebrec.models.newsrec.npa import NPAModel

MODEL_NAME = "NPA"
LOG_DIR = f"./runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"./runs/data/state_dict/{MODEL_NAME}/weights"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
modelcheckpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1)

config = hparams_npa
model = NPAModel(hparams=config, word2vec_embedding=word2vec_embedding, seed=42)
# model.model.summary()
hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    # epochs=1,
    callbacks=[tensorboard_callback, early_stopping_callback, modelcheckpoint_callback],
)
model.model.load_weights(filepath=MODEL_WEIGHTS)


2024-05-11 21:26:59.290054: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-05-11 21:26:59.290102: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-11 21:26:59.290115: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-11 21:26:59.290165: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-11 21:26:59.290180: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (64, 30) + inhomogeneous part.

In [ ]:
"""
evaluate performance
"""

pred_validation = model.scorer.predict(val_dataloader)

4/4 [==============================] - 2s 365ms/step


In [ ]:
df_validation = add_prediction_scores(df_validation, pred_validation.tolist()).pipe(
    add_known_user_column, known_users=df_train[DEFAULT_USER_COL]
)
df_validation.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,labels,scores,is_known_user
u32,list[i32],list[i32],list[i32],list[i8],list[f64],bool
2094919,"[9773351, 9774187, … 9779860]","[9780702, 9788188, … 9787499]",[9787499],"[0, 0, … 1]","[0.49845, 0.497381, … 0.497841]",false
2029201,"[9778326, 9777941, … 9779738]","[9783405, 9783852, … 8496358]",[9783852],"[0, 1, … 0]","[0.49801, 0.497147, … 0.498185]",false


In [ ]:
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore

metrics = MetricEvaluator(
    labels=df_validation["labels"].to_list(),
    predictions=df_validation["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

<MetricEvaluator class>: 
 {
    "auc": 0.4832519055489523,
    "mrr": 0.30877717899621926,
    "ndcg@5": 0.32656298038911985,
    "ndcg@10": 0.4381692958258077
}

In [ ]:
# pred_test = model.scorer.predict(test_dataloader) <--- breaks because of size mismatch
pred_test = model.scorer.predict(val_dataloader)

# store
submission_path = Path(os.getcwd()).parent / "submissions" / "ebnerd_lstur.txt"
with open(submission_path, "w") as f:
    for idx, row in enumerate(pred_test):
        f.write(f"{idx} {row}\n")
    f.close()

4/4 [==============================] - 3s 329ms/step
